In [1]:
import numpy as np
from scipy.linalg import qr

# Aufgabe 11: QR-Algorithmus

Implementieren Sie den QR-Algorithmus (Algorithmus 6.1 im Skript). Die Funktion `qr_algorithmus(A, tol, kmax)` erhält dazu die Matrix $A \in \R^{n \times n}$. Der Algorithmus soll nach `kmax` Iterationen abbrechen, oder falls
$$ \|s_k\|_2 \le \mathtt{tol}$$
wobei der Vektor $s_k$ die Subdiagonale $s_k = [(A_k)_{21}, (A_k)_{32}, \dots, (A_k)_{n,n-1}]$ darstellt.

Die Funktion soll die finale Matrix $A_k$ zurückgeben, die Matrix $Q$, sodass $A = Q A_k Q^T$, sowie die Anzahl der Iterationen $k$.


**Wichtiger Hinweis:** Sie dürfen die Funktion `qr` aus `scipy.linalg` (Import oben) für die QR-Zerlegung verwenden

In [2]:
def qr_algorithmus(A, tol, kmax):
    n = A.shape[0]
    k, tol, kmax = 0, abs(tol), abs(kmax)
    Amax = np.max(np.abs(A))
    Ak = np.copy(A) / Amax
    
    Q = np.eye(n)

    tolk = 2 * tol
    while k < kmax and tolk > tol:
        # Scipy QR is efficient for both Hessenberg and dense matrices
        Qk, Rk = qr(Ak)
        Ak, Q = Rk @ Qk, Q @ Qk
        # Stopping criterion: norm of subdiagonal
        tolk = np.linalg.norm(np.diag(Ak, k=-1))
        k += 1

    return Ak * Amax, Q, k

## Testskript

Ihre Funktion wird zuerst mit zwei Matrizen getestet.

In [3]:
import time
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def random_matrix(n):
    np.random.seed(1337)
    Q = ortho_group.rvs(n)
    return Q.T @ np.triu(np.random.randn(n, n)) @ Q

def laplace_matrix(n):
    h = float(n * n)
    return np.diag(np.full(n, 2.0*h)) - np.diag(np.full(n-1, h), -1) - np.diag(np.full(n-1, h), 1)

n_start = 5
kmax_start = 1000
num_tests = 3
matrices = [laplace_matrix, random_matrix]
matrix_names = ["Laplace-Matrix", "Zufalls-Matrix"]

total_time = 0.0

max_error = 0
for i in range(len(matrices)):
    n, kmax = n_start, kmax_start
    print(f"\n--- Testing: {matrix_names[i]} ---")
    
    for _ in range(num_tests):
        A = matrices[i](n)
        ref_eig = np.sort(np.linalg.eigvals(A))
        
        time_start = time.perf_counter()
        T, Q, k = qr_algorithmus(A.copy(), 1e-13, kmax)
        duration = time.perf_counter() - time_start
        
        # Validation
        err_reconstruction = np.max(np.abs(A - Q @ T @ Q.T))
        err_eigenvalues = np.max(np.abs(ref_eig - np.sort(np.diag(T))))
        max_error = max(max_error, err_reconstruction, err_eigenvalues)
        
        print(f"n={n:2d} | k={k:4d} | Rekonstruktionsfehler={err_reconstruction:.2e} | Eigenwertfehler={err_eigenvalues:.2e} | Zeit={duration:.4f}s")
        total_time += duration
        
        n, kmax = n*2, kmax*4

print(f"\nTotal Computation Time: {total_time:.4f}s")

assert max_error < 1e-8, f"Maximaler Fehler ist {max_error}. Beheben sie erst Ihre Fehler"


--- Testing: Laplace-Matrix ---
n= 5 | k= 136 | Rekonstruktionsfehler=1.77e-13 | Eigenwertfehler=4.97e-14 | Zeit=0.0023s
n=10 | k= 458 | Rekonstruktionsfehler=1.96e-12 | Eigenwertfehler=1.08e-12 | Zeit=0.0073s
n=20 | k=1614 | Rekonstruktionsfehler=2.10e-11 | Eigenwertfehler=8.19e-12 | Zeit=0.0333s

--- Testing: Zufalls-Matrix ---
n= 5 | k= 192 | Rekonstruktionsfehler=2.42e-14 | Eigenwertfehler=8.55e-14 | Zeit=0.0028s
n=10 | k= 546 | Rekonstruktionsfehler=3.24e-14 | Eigenwertfehler=2.50e-14 | Zeit=0.0090s
n=20 | k=9205 | Rekonstruktionsfehler=5.15e-13 | Eigenwertfehler=6.50e-09 | Zeit=0.7602s

Total Computation Time: 0.8149s


Hier entsteht noch eine kleine Animation, die die Konvergenz verdeutlichen soll.

In [5]:
n = 15
iterations = 20
matrix = random_matrix(n)
matrix_history = []
for i in range(iterations):
    matrix_history.append(qr_algorithmus(matrix.copy(), 1e-20, i)[0])

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(matrix_history[0], cmap='magma', interpolation='nearest')
plt.colorbar(im, ax=ax, label='Absolute Value')
ax.set_title(f"QR Algorithmus Konvergenz (n={n})")

def update(frame):
    im.set_array(matrix_history[frame])
    ax.set_xlabel(f"Iteration: {frame}")
    return [im]

# 4. Create and Display Animation
# Note: 'jshtml' is the best choice for JupyterLite/Lab
ani = FuncAnimation(fig, update, frames=iterations, interval=200, blit=True)
plt.close() # Prevents static plot from appearing
HTML(ani.to_jshtml())